### Data Preparation

Start by preparing the data:

0.  Prepare you data to fit the csv format
1.  Combine all csv files in one DataFrame
2.  Search for excitation frequencies and kick all measurements with f_min > f_min_required and f_max < f_max_required (no extrapolation)
3.  Calculate and filter by LinKK
4.  Add further informations that are necessary
5.  Interpolate all measurements, to have the same frequency points and fransform the DataFrame into its final shape
6.  Filter and clean your data, e.g. Temperature > 10000 °C
7.  Add information, like ECM fit, DRT fit and extrema in bode and nyquist diagram
8.  Save the DataFrame
9.  Evaluate the Data Preparation

In [ ]:
%matplotlib widget 

from modules import data_preparation as dp
from modules import eisplot as eisplot

import numpy as np
import pandas as pd

from impedance.models.circuits import CustomCircuit
from impedance.models.circuits.fitting import rmse

cm = 1/2.54  # centimeters in inches

## if you have installed latex and want to use it for plots, uncomment the following 2 lines
# eisplot.mpl.rcParams.update({"text.usetex": True,'savefig.format':'pdf'})
# eisplot.mpl.rc('text.latex', preamble=r'\usepackage{underscore}')

## safe figures e.g. with:
# plot_name = "custom_3D_plot"
# eisplot.plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".pdf")
# eisplot.plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".png", dpi=600)

### 0. Input CSV files

We assume that your EIS measurements for each cell are combined in one CSV file and the file name represents the cell name.  
The file/cell name later becomes the index column "cell_ID" in our DF.  
Besides the required values on the left in the table below, any additional parameter can be provided. Here e.g. temperature and SOC, but could also be SOH or any other.  
Put all files in a single folder.

EIS_measurement_id: &emsp; Integer counter, increase by 1 for each complete spectra  
EIS_Frequency:&emsp;&emsp;&emsp;&emsp;&nbsp;In Hertz  
EIS_Z_abs:&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;In Ohm  
EIS_Z_phase:&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;In radian

<style scoped>
table {
  font-size: 10px;
}
</style>

| EIS_measurement_id  | EIS_Frequency  | EIS_Z_abs  | EIS_Z_phase  | ...  | Temperature  | SOC      | ...  |
| ---                | ---            | ---         | ---           | ---   | ---           | ---       | ---   |
| 1                   | 0.01           | 0.02251    | 0.38192      | ...  | 25.03        | 80.01     | ...  |
| 1                   | 0.2304          | 0.02341    | 0.32001      | ...  | 25.11        | 80.01     | ...  |
| ...                 | ...            | ...        | ...          | ...  | ...          | ...      | ...  |
| 2                   | 0.01           | 0.02260    | 0.38246      | ...  | 24.89        | 60.13    | ...  |
| 2                   | 0.2304          | 0.02351    | 0.31981      | ...  | 25.02        | 60.13    | ...  |

### 1. Combine all csv files in one DataFrame

First we search all filenames that contain at least one of the strings from `search_filters`  
Furthermore we set a name for this run/evaluation

In [ ]:
name_of_this_run = "example_data"

folder_path = r"./example_input_data/"
search_filters = ["LiFun_575166-01"]
filenames = dp.search_filenames(folder_path, search_filters)
print(filenames)

Now we can read all these files into a single DataFrame:

In [ ]:
df = dp.get_df_folder(folder_path, filenames)
df = df.sort_values(by=['cell_ID', 'EIS_measurement_id', 'EIS_Frequency'])
df

### 2. Search for excitation frequencies used most often

In [ ]:
frequency_groups_df, frequency_groups = dp.get_frequency_groups(df)

Prefilter the frequency groups by obvious unwanted measurement:

$
\mathrm{Lower\ Frequency\ Boundary} \leq \mathrm{min(}\rm{EIS}_\mathrm{Frequency})
$  
$
\mathrm{Upper\ Frequency\ Boundary} \geq \mathrm{max(}\rm{EIS}_\mathrm{Frequency})
$  
$
\mathrm{Lower\ Boundary\ Frequencies\ Count} \leq \mathrm{length(}\rm{EIS}_\mathrm{Frequency} \mathrm{\ of\ one\ } \rm{EIS}_\mathrm{measurement\ id})
$

In [ ]:
lower_frequency_boundary = 10
upper_frequency_boundary = 1
lower_boundary_frequencies_count = 10


group_index_to_delte = []
group_index_to_delte = np.unique(np.concatenate(
    (frequency_groups_df.index[frequency_groups_df["min"]
                               >= lower_frequency_boundary].to_numpy(dtype=int), group_index_to_delte), axis=0))
group_index_to_delte = np.unique(np.concatenate(
    (frequency_groups_df.index[frequency_groups_df["max"]
                               <= upper_frequency_boundary].to_numpy(dtype=int), group_index_to_delte), axis=0))
group_index_to_delte = np.unique(np.concatenate(
    (frequency_groups_df.index[frequency_groups_df["length"]
                               <= lower_boundary_frequencies_count].to_numpy(dtype=int), group_index_to_delte), axis=0))
group_index_to_delte = group_index_to_delte.astype('int')
if len(group_index_to_delte) > 0:
    print("Delted: ", np.sum(frequency_groups_df.iloc[group_index_to_delte]['occurrence'].to_numpy()),
          " of ", np.sum(frequency_groups_df['occurrence'].to_numpy()), " EIS Measurements")
    frequency_groups_df = frequency_groups_df.drop(group_index_to_delte)
    frequency_groups = list(
        np.delete(np.array(frequency_groups, dtype='object'), group_index_to_delte))

Lets get an overview, later we will filter

In [ ]:
fig, axes = eisplot.plt.subplots(nrows=1, ncols=2, sharey=True, gridspec_kw={'width_ratios': [2, 1]}, figsize=(20*cm, 9*cm))
dp.plot_frequency_groups(fig, axes[0], axes[1], frequency_groups, df)

Now we define the final boundaries of the frequencies that should be used.

In [ ]:
fig, axes = eisplot.plt.subplots(1, 1, sharex=True, figsize=(20*cm, 9*cm))
fs_widget = dp.FrequencySelectWidget(axes, fig, df, frequency_groups_df, frequency_groups)

Extract the frequencies from the widget, or hardcode a custom set

In [ ]:
frequencies = fs_widget.frequencies
# frequencies = np.array([0.1,1,10,100,1000])
frequencies

Delete all measurements, that don't cover the chosen range of frequencies

In [ ]:
df = dp.filter_by_frequencies(df, frequencies, lower_boundary_frequencies_count)

### 3. Calculate and filter by LinKK

In [ ]:
df = dp.add_linKK(df, linKK_cutoff_mu=.02, linKK_max_M=8, residuals_quantile=1)

Choose a limit, e.g. < 100 mΩ

In [ ]:
fig, axes = eisplot.plt.subplots(nrows=1, ncols=2, sharey=True, gridspec_kw={'width_ratios': [2, 1]}, figsize=(20*cm, 9*cm))
linKK_widget = dp.LinkkSelectWidget(axes[0], axes[1], fig, df)

Apply the limit to the df

In [ ]:
measurements_before_delete = len(df.index.droplevel("EIS_Frequency").unique()) # For the sankey diagram
df = df[df['linKK'] <= linKK_widget.linKK_limit]
measurements_after_delete = len(df.index.droplevel("EIS_Frequency").unique())
dp.sankey_dict.update({'filter_by_linKK': measurements_after_delete-measurements_before_delete}) # For the sankey diagram

### 4. Add further informations that are necessary

For Nyquist diagrams or different feature setups, we may also need Cartesian coordinates in our dataset and do not want to recalculate them each time.

In [ ]:
df = dp.add_cartesian(df)

In [ ]:
df = df.sort_values(by=['cell_ID', 'EIS_measurement_id', 'EIS_Frequency'])

Maybe delete unwanted information

In [ ]:
# df = df.drop(columns=['Ah_Counter','Current','Duration','Ah_throughput','Wh_throughput','Capacity','Capacity_current','SOH','SOC'])

Or change some units

In [ ]:
df['SOC'] = df['SOC']*100
df['SOH'] = df['SOH']*100

### 5. Interpolate all measurements and fransform the DataFrame into its final shape

To align all measurements to the same frequecies we interpolate all measurements.  
Afterwards the table is transformed into its final shape.

<style scoped>
table {
  font-size: 10px;
}
</style>

| EIS_measurement_id  | EIS_Frequency   | EIS_Z_abs  | EIS_Z_phase  | ... | Temperature | SOC     | ...  |
| ---                 | ---             | ---        | ---          | --- | ---         | ---     | ---  |
| 1                   | 0.01            | 0.02251    | 0.38192      | ... | 25.03       | 80.01   | ...  |
| 1                   | 0.2304          | 0.02341    | 0.32001      | ... | 25.11       | 80.01   | ...  |
| ...                 | ...             | ...        | ...          | ... | ...         | ...     | ...  |
| 2                   | 0.01            | 0.02260    | 0.38246      | ... | 24.89       | 60.13   | ...  |
| 2                   | 0.2304          | 0.02351    | 0.31981      | ... | 25.02       | 60.13   | ...  |

↓

| cell_ID | EIS_measurement_id  | ...   | Temperature | SOC   | ...   | EIS_Z_abs_0.01  | EIS_Z_phase_0.01  | EIS_Z_Re_0.01 | EIS_Z_Im_0.01 | ...   |
| ---     | ---                 | ---   | ---         | ---   | ---   | ---             | ---               | ---           | ---           | ---   |
| A0000   | 1                   | ...   | 25.07       | 80.01 | ...   | 0.02251         | 0.38192           | 0.0209        | 0.0084        | ...   |
| A0000   | 2                   | ...   | 24.96       | 60.13 | ...   | 0.02260         | 0.38246           | 0.0226        | 0.0084        | ...   |

Comparison before and after the interpolation of one measurement

In [ ]:
# hide plots for now
eisplot.plt.ioff()

df_cell = df.loc[df.first_valid_index()[0]]
df_cell = df_cell.loc[df.first_valid_index()[1]]

fig, axes = eisplot.plt.subplots(nrows=1, ncols=1, figsize=(20*cm, 9*cm))
axes.plot(df_cell.EIS_Z_Re*1000, -df_cell.EIS_Z_Im*1000, marker='x')
axes.grid()
axes.set_aspect('equal', 'box')
if eisplot.mpl.rcParams['text.usetex'] == True:
    axes.set_xlabel(r"$\Re(\underline{Z})$ in m$\Omega$")
    axes.set_ylabel(r"$\Im(\underline{Z})$ in m$\Omega$")
else:
    axes.set_xlabel(r"Re(Z) in mΩ")
    axes.set_ylabel(r"Im(Z) in mΩ")
fig.subplots_adjust(bottom=0.14)

In [ ]:
df = dp.eis_interpolate_restruct(df, frequencies)

Save the column names of the EIS values

In [ ]:
destination_filepath = "data/key_lookup/key_lookup_"+name_of_this_run+".parquet"
eis_columns = ['EIS_Z_abs', 'EIS_Z_phase', 'EIS_Z_Re', 'EIS_Z_Im']
key_lookup_df = dp.get_key_lookup_df(eis_columns, frequencies)
key_lookup_df = key_lookup_df.reset_index()
key_lookup_df.to_parquet(destination_filepath, compression='gzip', index=True)

Compare before and after interpolation

In [ ]:
# show plots again
eisplot.plt.ion()

df_cell = df.loc[df.first_valid_index()[0]]
df_cell = df_cell.loc[df.first_valid_index()[1]]
Re_keys = key_lookup_df["EIS_Z_Re"].to_list()
Im_keys = key_lookup_df["EIS_Z_Im"].to_list()
z_real = df_cell[Re_keys]*1000
z_imag = df_cell[Im_keys]*1000
axes.plot(z_real, -z_imag, marker='x')
axes.legend(["before Interpolation", "after Interpolation"])

fig.set_visible(True)
fig.show()

### 6. Filter your data, e.g. Temperature > 10000 °C

In [ ]:
measurements_before_delte = len(df)  # For the sankey diagram

df = df.loc[df["Temperature"] < 100]
df = df.loc[df["Temperature"] > -60]

df = df.loc[df["Voltage"] >= -0.1]
df = df.loc[df["Voltage"] <= 100]

df = df.loc[df["SOC"] >= -20]
df = df.loc[df["SOC"] <= 120]

measurements_after_delte = len(df)  # For the sankey diagram
dp.sankey_dict.update({'filter_by_value': measurements_after_delte-measurements_before_delte})

In [ ]:
fig, axes = eisplot.plt.subplots(1, 2, figsize=(20*cm, 9*cm))

axes[0].hist(df["Temperature"].to_numpy(), density=True, bins=25)
axes[0].set_yscale('log')
axes[0].grid(True)

axes[1].hist(df["SOC"].to_numpy(), density=True, bins=106)
axes[1].set_yscale('log')
axes[1].grid(True)

axes[0].set_ylabel("Normalized probability")
axes[0].set_xlabel("Temperature in $^\circ$C")
axes[1].set_xlabel("SoC in $\%$")

fig.subplots_adjust(bottom=0.14)

### 7. Add information, like ECM fit, DRT fit and extrema

Add ECM fit parameters with [impedance.py](https://impedancepy.readthedocs.io/en/latest/)

<style scoped>
table {
  font-size: 10px;
}
</style>

| cell_ID | EIS_measurement_id  | ...  |ECM_R0 | ... | ECM_W3 | ECM_Fit_RMSE  |
| ---     | ---                 | ---  | ---   | --- | ---    | ---           |
| A0000   | 1                   | ...  | AAA   | ... | BBB    | ...           |
| A0000   | 2                   | ...  | CCC   | ... | DDD    | ...           |

Start with visual inspection of a spectra

In [ ]:
df_cell = df[np.in1d(df.index.get_level_values(0),[df.first_valid_index()[0]])]

fig, axes, cmap = eisplot.plot_nyquist_feature(df_cell, key_lookup_df, feature="Temperature")

Continue with the fit of one measurement

In [ ]:
circuit = 'L0-R0-p(CPE1,R1)-p(CPE2,R2)-p(CPE3,R3)'
initial_guess = [1.0e-08,
                 1.0e-02,
                 1.0e-01, 0.5e+00,  1.0e-03,
                 1.0e+01, 0.5e+00,  1.0e-01,
                 1.0e+03, 0.5e+00,  1.0e+01]
c_circuit = CustomCircuit(circuit, initial_guess=initial_guess)

In [ ]:
abs_keys = key_lookup_df["EIS_Z_abs"].to_list()
phase_keys = key_lookup_df["EIS_Z_phase"].to_list()
frequencies = key_lookup_df["frequency"].to_numpy()

# important to use dtype='float64'
abs_values = df[abs_keys].to_numpy(dtype='float64')
phases = df[phase_keys].to_numpy(dtype='float64')
impedance_z = abs_values * np.exp(1j * phases)
impedance_z = impedance_z.astype(complex)
impedance_z = impedance_z[-1]

In [ ]:
c_circuit.fit(frequencies, impedance_z, global_opt=False)

impedance_z_fit = c_circuit.predict(frequencies)
impedance_z_fit_rmse = rmse(impedance_z, impedance_z_fit)

In [ ]:
fig, axes = eisplot.plt.subplots(1, 1, figsize=(20*cm, 9*cm))
axes.plot(np.real(impedance_z)*1000, -np.imag(impedance_z)*1000, linewidth=5.0)
axes.plot(np.real(impedance_z_fit)*1000, -np.imag(impedance_z_fit)* 1000, linestyle=':', linewidth=2.0)
axes.axis('equal')
axes.grid()
if eisplot.mpl.rcParams['text.usetex'] == True:
    axes.set_xlabel(r"$\Re(\underline{Z})$ in m$\Omega$")
    axes.set_ylabel(r"$\Im(\underline{Z})$ in m$\Omega$")
else:
    axes.set_xlabel(r"Re(Z) in mΩ")
    axes.set_ylabel(r"Im(Z) in mΩ")
axes.legend(["Measurement", "Fit"])
fig.subplots_adjust(bottom=0.14)

Add the chosen fit now to all measurements

In [ ]:
df = dp.add_ECMfit(df, key_lookup_df, circuit=circuit, initial_guess=initial_guess, maxfev=4000)
# df.plot(y="ECM_Fit_RMSE", xticks=[], xlabel='')

Calculate the DRT with [pyDRTtools](https://github.com/ciuccislab/pyDRTtools) and add the extrema of it to the table.  
This is added as a submodule in git, make sure, that you updated it with:  ``` git submodule update ```  
With ```drt_plot = True``` the code runs without parallelization  
Up to 9 peaks are added to the df, when fewer peaks are found the remaining values are set to NaN  

<style scoped>
table {
  font-size: 10px;
}
</style>

| cell_ID | EIS_measurement_id  | ...  | DRT_Peak_0_tau | DRT_Peak_0_gamma  | ...   | DRT_Peak_9_tau  | DRT_Peak_9_gamma  |
| ---     | ---                 | ---  | ---            | ---               | ---   | ---             | ---               |
| A0000   | 1                   | ...  | AAA            | BBB               | ...   | CCC             | DDD               |
| A0000   | 2                   | ...  | EEE            | FFF               | ...   | NaN             | NaN               |

Plot the DRT for e.g. 5 measurements

In [ ]:
# # grab a cell by its name
# cell_name = "LiFun_575166-01_002"
# df_cell = df.loc[df.index.get_level_values('cell_ID') == cell_name]
# # or just grab the first one
# df_cell = df[np.in1d(df.index.get_level_values(0), [df.first_valid_index()[0]])]
# # or sample e.g. 5 measurements
df_cell = df.sample(n=5)
[fig, axes, df_cell_drt_plot] = dp.add_DRT(df_cell, key_lookup_df, drt_plot=True, feature="Voltage")

Now add the DRT to all measurements without plotting

In [ ]:
df = dp.add_DRT(df, key_lookup_df, drt_plot=False)

Add extrema (min, max, zerocrossing) of the Nyquist and Bode diagrams  
Up to 3 points of each type can be added, when fewer found the remaining values are set to NaN

<style scoped>
table {
  font-size: 10px;
}
</style>

| cell_ID | EIS_measurement_id  | ...  | Nyquist_Zero_Real_Freq_0  | Nyquist_Zero_Real_Value_0  | ...   | Bode_Phase_Max_Freq_2   | Bode_Phase_Max_Value_2  |
| ---     | ---                 | ---  | ---                       | ---                        | ---   | ---                     | ---                     |
| A0000   | 1                   | ...  | AAA                       | BBB                        | ...   | CCC                     | DDD                     |
| A0000   | 2                   | ...  | EEE                       | FFF                        | ...   | NaN                     | NaN                     |

In [ ]:
df = dp.add_EIS_Extrema(df, key_lookup_df)

### 8. Save the DataFrame

In [ ]:
destination_filepath = r"./data/eis_datasets/"+name_of_this_run+".parquet"
df.to_parquet(destination_filepath, compression='gzip', index=True)

Verify if it was successful

In [ ]:
destination_filepath = r"./data/eis_datasets/"+name_of_this_run+".parquet"
df_in = pd.read_parquet(destination_filepath)

In [ ]:
np.sum(np.sum((df.notna() == df_in.notna()) == False))

### 9. Evaluate the Data Preparation

In [ ]:
dp.sankey_dict

In [ ]:
fig = eisplot.plt.figure(figsize=(8 * cm, 6 * cm))
ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])
sankey = eisplot.Sankey(ax=ax, scale=0.2/dp.sankey_dict['all_files_in'],
                offset=0.25, head_angle=45, format='%.0f', unit='')
sankey.add(flows=[dp.sankey_dict['all_files_in'], -dp.sankey_dict['filtered_files_in'],
                  -(dp.sankey_dict['all_files_in']-dp.sankey_dict['filtered_files_in'])],
           labels=['', 'used files', 'not used files'],
           orientations=[0, 0, 1])
ax.axis('off')
diagrams = sankey.finish()

In [ ]:
m_in = dp.sankey_dict['all_measurements_in']
m_out = m_in + dp.sankey_dict['filter_by_frequencies'] + \
    dp.sankey_dict['filter_by_linKK']+dp.sankey_dict['filter_by_value']

fig = eisplot.plt.figure(figsize=(8 * cm, 6 * cm))
ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])
sankey = eisplot.Sankey(ax=ax, scale=0.2/m_in, offset=0.25,
                head_angle=45, format='%.0f', unit='')
sankey.add(flows=[m_in, -m_out, dp.sankey_dict['filter_by_frequencies'],
                  dp.sankey_dict['filter_by_linKK'], dp.sankey_dict['filter_by_value']],
           labels=['', 'used measurements', 'Frequency', 'linKK', 'Value'],
           orientations=[0, 0, 1, -1, -1])
ax.axis('off')
diagrams = sankey.finish()